# BERT

Sources:
https://towardsdatascience.com/checking-grammar-with-bert-and-ulmfit-1f59c718fe75
https://gist.github.com/sayakmisra/dbb06efec99e760cf9e5d197175ad9c5#file-grammar-checker-bert-ipynb

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


Package from: https://github.com/huggingface/transformers

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 32.2 MB/s 
     |████████████████████████████████| 163 kB 29.9 MB/s 


# Loading Test Data

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# To unmount your Google Drive:
# drive.flush_and_unmount()

In [ ]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Erdos Fall 2022/Dataset/test.csv")

In [ ]:
# Report the number of essays in test set.
print('Number of test essays: {:,}\n'.format(df.shape[0]))

# Display
df

Number of test essays: 3



,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [ ]:
text = df['full_text'].apply(lambda x: x.replace('\r\n\r\n', ' ') and x.replace('\n\n', ' '))

# Import Grammar Checker BERT Model

In [ ]:
!pip install transformers

from transformers import BertForSequenceClassification

output_dir = "/content/drive/MyDrive/Colab Notebooks/Erdos Fall 2022/model_save/"

print(output_dir)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/Colab Notebooks/Erdos Fall 2022/model_save/


In [ ]:
from transformers import BertTokenizer
import torch
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(output_dir)
model_loaded = BertForSequenceClassification.from_pretrained(output_dir)

Loading BERT tokenizer...


# Create Grammar Dataframe for Test Set

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
from nltk import tokenize

In [ ]:
grammar_correct_ratio = []
sentence_number = []

In [ ]:
for i in range(len(text)):
  print('Running on essay ', i+1, '/',len(text))

  sentences = [sentence for sentence in tokenize.sent_tokenize(text[i])]
  encoded_dict = tokenizer.batch_encode_plus(
                          sentences,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 64,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
  # Add the encoded sentence to the list.    
  input_id = encoded_dict['input_ids']
      
  # And its attention mask (simply differentiates padding from non-padding).
  attention_mask = encoded_dict['attention_mask']
  input_id = torch.LongTensor(input_id)
  attention_mask = torch.LongTensor(attention_mask)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model_loaded = model_loaded.to(device)
  input_id = input_id.to(device)
  attention_mask = attention_mask.to(device)

  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model_loaded(input_id, token_type_ids=None, attention_mask=attention_mask)

  logits = outputs[0]
  index = logits.argmax(dim=1)

  sentence_number.append(len(sentences))
  grammar_correct_ratio.append(torch.sum(index).item()/len(sentences))

Running on essay  1 / 3
Running on essay  2 / 3


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Running on essay  3 / 3


In [ ]:
sentence_number

[26, 17, 17]

In [ ]:
# check the list of ratio
grammar_correct_ratio

[0.07692307692307693, 0.29411764705882354, 0.6470588235294118]

In [ ]:
df_grammar = pd.DataFrame({'cleaned_full_text':text, 'sentence_number': sentence_number, 'ratio_grammar_correct_sentences': grammar_correct_ratio })

In [ ]:
df_grammar

,cleaned_full_text,sentence_number,ratio_grammar_correct_sentences
0,when a person has no experience on a job their...,26,0.076923
1,Do you think students would benefit from being...,17,0.294118
2,"Thomas Jefferson once states that ""it is wonde...",17,0.647059


In [ ]:
# Save data to csv in Google Drive
df_grammar.to_csv('/content/drive/MyDrive/Colab Notebooks/Erdos Fall 2022/grammar_test.csv')